In [2]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [104]:
nsd_dir = '/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/'
fs_dir = '/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'
# inputs
subj = utils.sub_number_to_string(1, dataset='nsdsyn')
hemi = 'lh'
design_mat = os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', subj, 'nativesurface', prep._get_beta_folder_name(3),
                     f'{hemi}.betas_nsdsynthetic.hdf5')
stim_info = os.path.join(nsd_dir, 'nsdsynthetic_sf_stim_description.csv')
rois = [os.path.join(fs_dir, subj, 'label', f'{hemi}.prf-{roi}.mgz')
        for roi in ["visualrois", "eccrois"]]
prfs = [os.path.join(fs_dir, subj, 'label', f'{hemi}.prf{prf_param}.mgz')
        for prf_param in ["eccentricity", "angle", "size"]]


In [10]:
stim_df = prep.load_stim_info(stim_description_path=stim_info,
                              drop_phase=False)
stim_df

,image_idx,names,names_idx,w_r,w_a,phase,class_idx,freq_lvl,fixation_task,memory_task
0,104,pinwheel,0,0.0,6.0,0.000000,0,0,552,87
1,105,pinwheel,0,0.0,6.0,1.570796,0,0,208,673
2,106,pinwheel,0,0.0,6.0,3.141593,0,0,356,635
3,107,pinwheel,0,0.0,6.0,4.712389,0,0,152,431
4,108,pinwheel,0,0.0,11.0,0.000000,1,1,14,479
...,...,...,...,...,...,...,...,...,...,...
107,211,intermediate 3,6,34.0,-14.0,4.712389,26,3,402,123
108,212,intermediate 4,7,14.0,-34.0,0.000000,27,3,366,645
109,213,intermediate 4,7,14.0,-34.0,1.570796,27,3,704,239
110,214,intermediate 4,7,14.0,-34.0,3.141593,27,3,250,715


In [250]:
mask, rois_mgzs = prep.load_common_mask_and_rois(rois, [[1,2,3,4,5,6,7],[1,2,3,4,5]])

In [249]:
prf_mgzs = prep.load_prf_properties(prfs, mask)

In [233]:
betas_mgzs = prep.load_betas(betas, design_mat, stim_df['image_idx'], mask, task_list=['fixation_task','memory_task'], average=True)

In [245]:
betas_df = prep.melt_2D_beta_mgzs_into_df(betas_mgzs, x_axis='voxel', y_axis='stim_idx', long_format=True)

In [246]:
betas_df

,voxel,stim_idx,task,betas
0,0,0,fixation,1.966667
1,0,1,fixation,1.096667
2,0,2,fixation,2.103333
3,0,3,fixation,1.630000
4,0,4,fixation,2.940000
...,...,...,...,...
3666091,10910,107,avg,2.618333
3666092,10910,108,avg,2.253333
3666093,10910,109,avg,1.633333
3666094,10910,110,avg,1.805000


In [251]:
df = prep.add_prf_columns_to_df(prf_mgzs, betas_df, rois_mgzs, on='voxel')

In [252]:
df

,voxel,stim_idx,task,betas,lh.eccentricity,lh.angle,lh.size,lh.visualrois,lh.eccrois
0,0,0,fixation,1.966667,1.370705,292.141510,84.840942,V1,3.0
1,0,1,fixation,1.096667,1.370705,292.141510,84.840942,V1,3.0
2,0,2,fixation,2.103333,1.370705,292.141510,84.840942,V1,3.0
3,0,3,fixation,1.630000,1.370705,292.141510,84.840942,V1,3.0
4,0,4,fixation,2.940000,1.370705,292.141510,84.840942,V1,3.0
...,...,...,...,...,...,...,...,...,...
3666091,10910,107,avg,2.618333,6.167487,32.136745,1.082151,None,5.0
3666092,10910,108,avg,2.253333,6.167487,32.136745,1.082151,None,5.0
3666093,10910,109,avg,1.633333,6.167487,32.136745,1.082151,None,5.0
3666094,10910,110,avg,1.805000,6.167487,32.136745,1.082151,None,5.0
